<h1> 2c. Refactoring to add batching and feature-creation </h1>

In this notebook, we continue reading the same small dataset, but refactor our ML pipeline in two small, but significant, ways:
<ol>
<li> Refactor the input to read data in batches.
<li> Refactor the feature creation so that it is not one-to-one with inputs.
</ol>

In [1]:
import google.datalab.ml as ml
import tensorflow as tf
from tensorflow.contrib import layers
print tf.__version__
#print ml.sdk_location

1.4.1


In [2]:
import datalab.bigquery as bq
import tensorflow as tf
import numpy as np
import shutil

<h2> 1. Refactor the input </h2>

Read data created in Lab1a, but this time make it more general, so that we are reading in batches.  Instead of using Pandas, we will add a filename queue to the TensorFlow graph.  This queue will be cycled through num_epochs times.

In [3]:
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
LABEL_COLUMN = 'fare_amount'
DEFAULTS = [[0.0], [-74.0], [40.0], [-74.0], [40.7], [1.0], ['nokey']]

def read_dataset(filename, num_epochs=None, batch_size=512, mode=tf.contrib.learn.ModeKeys.TRAIN):
  def _input_fn():
    input_file_names = tf.train.match_filenames_once(filename)
    filename_queue = tf.train.string_input_producer(
        input_file_names, num_epochs=num_epochs, shuffle=True)
    reader = tf.TextLineReader()
    _, value = reader.read_up_to(filename_queue, num_records=batch_size)

    value_column = tf.expand_dims(value, -1)
    columns = tf.decode_csv(value_column, record_defaults=DEFAULTS)
    features = dict(zip(CSV_COLUMNS, columns))
    label = features.pop(LABEL_COLUMN)
    return features, label

  return _input_fn

def get_train():
  return read_dataset('./taxi-train.csv', num_epochs=100, mode=tf.contrib.learn.ModeKeys.TRAIN)

def get_valid():
  return read_dataset('./taxi-valid.csv', num_epochs=1, mode=tf.contrib.learn.ModeKeys.EVAL)

def get_test():
  return read_dataset('./taxi-test.csv', num_epochs=1, mode=tf.contrib.learn.ModeKeys.EVAL)

<h2> 2. Refactor the way features are created. </h2>

For now, pass these through (same as previous lab).  However, refactoring this way will enable us to break the one-to-one relationship between inputs and features.

In [4]:
INPUT_COLUMNS = [
    layers.real_valued_column('pickuplon'),
    layers.real_valued_column('pickuplat'),
    layers.real_valued_column('dropofflat'),
    layers.real_valued_column('dropofflon'),
    layers.real_valued_column('passengers'),
]

feature_cols = INPUT_COLUMNS

<h2> Create and train the model </h2>

Note that we no longer have a num_steps variable.  get_train() specifies a num_epochs.

In [7]:
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree('taxi_trained', ignore_errors=True) # start fresh each time
model = tf.estimator.LinearRegressor(
      feature_columns=feature_cols, model_dir='taxi_trained')
model.train(input_fn=get_train());

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c1c414fd0>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'taxi_trained', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:loss = 105246.0, step = 1
INFO:tensorflow:global_step/sec: 219.716
INFO:tensorflow:loss = 42109.7, step = 101 (0.456 sec)
INFO:tensorflow:global_step/sec: 236.424
INFO:tensorflow:loss = 41853.3, step = 201 (0.423 sec)
INFO:tensorflow:global_step/sec: 221.554
INFO:tensorflow:loss = 36080.0, step 

<h3> Evaluate model </h3>

As before, evaluate on the validation data.  We'll do the third refactoring (to move the evaluation into the training loop) in the next lab.

In [8]:
def print_rmse(model, name, input_fn):
  metrics = model.evaluate(input_fn=input_fn, steps=1)
  print 'RMSE on {} dataset = {}'.format(name, np.sqrt(metrics['loss']))
print_rmse(model, 'validation', get_valid())

INFO:tensorflow:Starting evaluation at 2018-02-26-10:36:15
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-1600
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2018-02-26-10:36:16
INFO:tensorflow:Saving dict for global step 1600: average_loss = 127.754, global_step = 1600, loss = 65410.2
RMSE on validation dataset = 255.754211426


Copyright 2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License